In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [42]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6,
    WL_total=2.4,
    WL_residual=2.2
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7,
    WL_total=-1.7,
    WL_residual=-1.4
)


## Parameters

In [43]:
# General constants
g = 9.81 # m/s^2
D = 9 # m (Diameter width)
rho_w = 1025 # kg/m^3
rho_s = 2650 # kg/m^3
bodemdiepte = 40 # mMSL
Delta_v = (rho_s - rho_w) / rho_w # Dimensionless

## Armour Layer
### Boek methode

Diameter van de armour layer wordt hieronder bepaald via de methode boek.
- De waarde van Kv moet nog geverifieerd worden. 

In [ ]:
def calculate_d_n50(u_c, H, T, h, psi_c, k_r, K_v, K_s=1.0, delta=Delta_v):
    """
    Calculate the characteristic stone diameter d_n50 for an armour layer (with adapted Shields formula).

    Parameters 
    ----------
    u_c : float
        Current velocity depth-averaged [m/s]
    H : float
        Wave height [m]
    T : float
        Wave period [s]
    h : float
        Water depth [m]
    psi_c : float, optional
        Critical motion parameter [-]
    k_r : float
        bottom_roughness [-]
    K_s : float, optional
        Strength reduction factor for stones on a slope [-]
    K_v : float, optional
        Velocity/turbulence factor [-]
    delta : float, optional
        Relative density (Δ = (rho_s - rho_w) / rho_w) [-]

    Returns
    -------
    d_n50 : float
        Required median stone diameter [m]
    """
    C = 50.0  # initial Chezy coefficient guess
    omega = 2.0 * np.pi / T
    k = omega**2 / g  # Initial guess for wave number
    for _ in range(10):
        k = omega**2 / (g * np.tanh(k * h))  # Update k using dispersion relation
    
    a_b = (H / 2.0) / np.sinh(k * h)
    u_b_hat = omega * a_b
    c_f = 0.237 * (a_b / k_r)**(-0.52)
    
    for _ in range(10):
        u_star_r_mean = np.sqrt((g / C**2) * u_c**2 + (c_f / 2) * u_b_hat**2 * 0.5)
        d_n50 = u_star_r_mean**2 / (psi_c * delta * g)
        C = 18 * np.log10(12 * D / (2.5 * d_n50))  # Update Chezy coefficient based on d_n50

    d_n50 = (K_v**2 * u_star_r_mean**2) / (K_s * psi_c * delta * g)

    print('kd=', k*h)

    return {'a_b': a_b, 'd_n50': d_n50, 'u_b_hat': u_b_hat, 'c_f': c_f, 'u_star_r_mean': u_star_r_mean}



kd= 0.9712397536672275
kd= 1.0811631966426112
For HWL conditions:  
	- u_c = 0.5 m/s,
	- H = 9.9 m,
	- T = 15.3 s,
	- h = 42.4 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.2192 m/s
 and the required median stone diameter is:	 d_n50 = 2.474432814777331 cm.

For LWL conditions:  
	- u_c = 0.5 m/s,
	- H = 8.7 m,
	- T = 13.4 s,
	- h = 38.3 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.2049 m/s
 and the required median stone diameter is:	 d_n50 = 2.160895131866744 cm.
0.0588 0.0677


In [59]:
def calculate_d_n50(u_c, H, T, h, psi_c, k_r, K_v, K_s=1.0, delta=Delta_v, phi=np.pi/2):
    """
    Calculate characteristic stone diameter using time-averaged
    and maximum combined wave-current shear velocity.
    """

    # --- Constants
    C_mean = 50.0
    C_max = 50.0
    omega = 2.0 * np.pi / T

    # --- Wave number (dispersion relation)
    k = omega**2 / g
    for _ in range(10):
        k = omega**2 / (g * np.tanh(k * h))

    # --- Orbital velocity at bed
    a_b = (H / 2.0) / np.sinh(k * h)
    u_b_hat = omega * a_b

    # --- Wave friction factor
    c_f = 0.237 * (a_b / k_r) ** (-0.52)

    # --- Iteration for Chezy–d50 coupling
    for _ in range(10):

        # Time-averaged u*_r
        u_star_r_mean = np.sqrt(
            (g / C_mean**2) * u_c**2 +
            (c_f / 4.0) * u_b_hat**2
        )

        # Maximum u*_r (sin(ωt)=1)
        u_star_r_max = np.sqrt(
            (g / C_max**2) * u_c**2 +
            (c_f / 2.0) * u_b_hat**2 +
            2.0 * np.sqrt(g) / C_max * u_c * np.sqrt(c_f / 2.0) * u_b_hat * np.sin(phi)
        )

        # Update stone size based on mean shear
        d_n50_mean = u_star_r_mean**2 / (psi_c * delta * g)
        d_n50_max = u_star_r_max**2 / (psi_c * delta * g)

        # Update Chezy
        C_mean = 18.0 * np.log10(12.0 * D / (2.5 * d_n50_mean))
        C_max = 18.0 * np.log10(12.0 * D / (2.5 * d_n50_max))

    # --- Design value including safety factors
    d_n50_mean = (K_v**2 * u_star_r_mean**2) / (K_s * psi_c * delta * g)
    d_n50_max = (K_v**2 * u_star_r_max**2) / (K_s * psi_c * delta * g)

    return {
        'a_b': a_b,
        'u_b_hat': u_b_hat,
        'c_f': c_f,
        'u_star_r_mean': u_star_r_mean,
        'u_star_r_max': u_star_r_max,
        'd_n50_mean': d_n50_mean,
        'd_n50_max': d_n50_max
    }
    
    
Kv = 1.55 # Velocity coefficient
shields_parameter = 0.3
roughtnes_bottem = 0.3

res_HWL = calculate_d_n50(u_c=HWL.CS_total, H=HWL.Hm0, T=HWL.Tp, h=HWL.WL_total + bodemdiepte, K_v=Kv, psi_c=shields_parameter, k_r=roughtnes_bottem)
res_LWL = calculate_d_n50(u_c=LWL.CS_total, H=LWL.Hm0, T=LWL.Tp, h=LWL.WL_total + bodemdiepte, K_v=Kv, psi_c=shields_parameter, k_r=roughtnes_bottem)

print(f'For HWL conditions:  \n\t- u_c = {HWL.CS_total} m/s,\n\t- H = {HWL.Hm0} m,\n\t- T = {HWL.Tp} s,\n\t- h = {HWL.WL_total + bodemdiepte} m\nThe resulting time-averaged shear velocity:\t u_star_r_mean = {res_HWL["u_star_r_mean"]:.4f} m/s & u_star_r_max = {res_HWL["u_star_r_max"]:.4f} m/s\n and the required median stone diameter is:\t d_n50 = {res_HWL["d_n50_mean"] * 100:.2f} cm (with mean shear) or d_n50 = {res_HWL["d_n50_max"] * 100:.2f} (with max shear).\n')

For HWL conditions:  
	- u_c = 0.5 m/s,
	- H = 9.9 m,
	- T = 15.3 s,
	- h = 42.4 m
The resulting time-averaged shear velocity:	 u_star_r_mean = 0.2192 m/s & u_star_r_max = 0.3349 m/s
 and the required median stone diameter is:	 d_n50 = 2.47 cm (with mean shear) or d_n50 = 5.77 (with max shear).

